# Transfer Learning

We will use pre-trained networks to solve challenging problems in computer vision. Specifically, we will use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/stable/torchvision/models.html).

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It is used to train deep neural networks using an architecture called convolutional layers.

Once trained, these models work astonishingly well as feature detectors for images they were not trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we will use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your application. We will include `models` in our imports now.

In [24]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. We also need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`. 

In [25]:
# Uncomment these lines to download and unzip files as  directory
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip Cat_Dog_data.zip

--2020-11-28 22:37:47--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.100.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.100.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580495262 (554M) [application/zip]
Saving to: ‘Cat_Dog_data.zip.1’

Cat_Dog_data.zip.1  100%[===================>] 553.60M  46.7MB/s    in 12s     

2020-11-28 22:37:59 (44.7 MB/s) - ‘Cat_Dog_data.zip.1’ saved [580495262/580495262]

Archive:  Cat_Dog_data.zip
replace Cat_Dog_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Cat_Dog_data/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Cat_Dog_data/test/cat/cat.7660.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [26]:
data_dir = 'Cat_Dog_data'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define Transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here
train_data = datasets.ImageFolder(data_dir, transform=train_transforms)
test_data = datasets.ImageFolder(data_dir, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, 
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as `DenseNet`. Let's print out the model architecture so we can see what is going on.

In [27]:
model = models.densenet121(pretrained=True).to(device)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it will not work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, pre-trained networks can be viewed as good feature detectors that can be used as the input for simple feed-forward classifiers.

In [28]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
  param.requires_grad = False

from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.classifier = classifier

With our model built, we need to train the classifier. However, now we are using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we are going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It is also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with most other deep learning frameworks, uses [CUDA]() to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you will commonly fo when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I will compare how long it takes to perform a forward and backward pass with and without a GPU.

In [29]:
import time

for device in ['cpu', 'cuda']:
  criterion = nn.NLLLoss()

  # Only train the classifier parameters, feature parameters are frozen
  optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

  model.to(device)

  for ii, (inputs, labels) in enumerate(trainloader):

    # Move input and label tensors to the GPU
    inputs, labels = inputs.to(device), labels.to(device)

    start = time.time()

    outputs = model.forward(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if ii==3:
      break
  
  print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 4.688 seconds
Device = cuda; Time per batch: 0.010 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:

```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, you will finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained model to classify the cat and dog images. Continue with the DenseNet model, or try ResNet which is a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [ ]:
## TODO: Use a pretrained model to classify the cat and dog images

criterion = nn.NLLLoss().to(device)

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

epochs = 10
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:

    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    log_ps = model(images)
    loss = criterion(log_ps, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  else: # after for-loop above completes, execute this code below
    test_loss = 0
    accuracy = 0

    ## TODO: Implement the validation pass and print out the validation accuracy
    with torch.no_grad():
      for test_images, test_labels in testloader:
        test_images, test_labels = test_images.to(device), test_labels.to(device)
        log_ps = model(test_images)
        test_loss += criterion(log_ps, test_labels)

        ps = torch.exp(log_ps)
        _, top_class = ps.topk(1, dim=1)
        equals = top_class == test_labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

    train_losses.append(running_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))

    print("Epoch: {}/{}.. ".format(e+1, epochs),
          "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
          "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))


Epoch: 1/10..  Training Loss: 0.333..  Test Loss: 0.331..  Test Accuracy: 0.900
Epoch: 2/10..  Training Loss: 0.329..  Test Loss: 0.328..  Test Accuracy: 0.900
